## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = range(1, 1001)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [5]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [7]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [8]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [9]:
# count
rdd.count()

1000

In [10]:
# first
rdd.first()

1

In [12]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [14]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [15]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([8.43939827e-02, 1.38266499e+00, 2.29225968e+00, 2.89937157e+00,
       9.47722179e-01, 4.06893075e+00, 4.14623487e+00, 2.47146473e+00,
       2.45179778e+00, 1.94857924e+00, 1.07929665e+01, 1.65072182e+00,
       6.81041263e+00, 2.33883084e+00, 1.25158967e+01, 1.23371325e+01,
       2.45307765e+00, 5.27459635e+00, 8.88887079e+00, 6.23292271e+00,
       7.65656404e+00, 4.37812150e+00, 1.97547275e+01, 2.27547413e+01,
       1.38100706e+01, 2.35218545e+01, 6.31199326e-02, 1.67414738e+01,
       1.84527329e+01, 1.96548568e+01, 2.73325896e+01, 1.94004446e+01,
       1.79150647e+00, 1.82304592e+01, 4.48642862e+00, 2.20108877e+01,
       2.91696071e+01, 7.47092789e+00, 1.42084320e+01, 8.78274871e+00,
       3.90615315e+01, 2.60612130e+00, 2.51267099e+01, 1.78593559e+01,
       4.30396440e+01, 1.39028099e+01, 3.09617724e+01, 2.26609284e+01,
       3.73484895e+01, 5.56303979e+00, 1.42264441e+01, 4.87547839e+01,
       3.92091012e+01, 4.25368395e+01, 3.07178067e+01, 2.22514941e+00,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [17]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.top(5)

[984.5249826506714,
 920.7296448380582,
 894.5975034594666,
 889.0490125289831,
 883.4545069505739]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [20]:
def sales_tax(num):
    return num - (num * 0.08)

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [25]:
# perform action to retrieve rdd values
revenue_minus_tax.take(20)

[0.07764246407782886,
 1.2720517908066613,
 2.108878904070078,
 2.6674218456609364,
 0.8719044042545593,
 3.7434162932388473,
 3.8145360769298655,
 2.273747554259933,
 2.2556539613591573,
 1.792692897550425,
 9.929529191684974,
 1.518664070961356,
 6.265579615596369,
 2.1517243733283142,
 11.514624919069016,
 11.350161892749867,
 2.256831436712811,
 4.852628637644486,
 8.177761125595225,
 5.734288892664282]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [26]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [27]:
discounted.take(10)

[0.06987821767004598,
 1.1448466117259952,
 1.8979910136630704,
 2.4006796610948427,
 0.7847139638291034,
 3.3690746639149625,
 3.433082469236879,
 2.0463727988339397,
 2.0300885652232417,
 1.6134236077953825]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [28]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[815.1866856347559,
 762.3641459259122,
 740.7267328644383,
 736.132582373998,
 731.5003317550752,
 720.461750580002,
 717.3707889601844,
 708.5690312786207,
 705.4674991928019,
 705.412187096336,
 701.7051048423474,
 699.7137629443749,
 697.983572107377,
 695.6114275884222,
 687.6667799904149]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [29]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [30]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.08439398269329224, 0.06987821767004598), (1.3826649900072405, 1.1448466117259952), (2.2922596783370413, 1.8979910136630704), (2.899371571370583, 2.400679661094843), (0.9477221785375645, 0.7847139638291035), (4.068930753520486, 3.3690746639149625), (4.146234866228115, 3.4330824692368793), (2.4714647328912314, 2.0463727988339397), (2.4517977840860405, 2.0300885652232417), (1.9485792364678534, 1.6134236077953827)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [31]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.08439398269329224, 0.06987821767004598, 1.3826649900072405, 1.1448466117259952, 2.2922596783370413, 1.8979910136630704, 2.899371571370583, 2.400679661094843, 0.9477221785375645, 0.7847139638291035]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [32]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()


280

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [33]:
selected_items.reduce(lambda x,y: x + y)

129556.48055851093

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [34]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(38, 304.5836698103771),
 (38, 315.56507863194776),
 (47, 317.9956498143488),
 (38, 339.2203105375484),
 (50, 321.04107906213153),
 (10, 356.08226889776165),
 (26, 304.7474334684345)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [35]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(50, 7294.908013760045),
 (10, 2821.334711698309),
 (20, 2483.5366373268894),
 (40, 1791.5655048450471),
 (30, 1454.32432229845),
 (41, 4678.662064399374),
 (31, 9077.582354244241),
 (21, 2220.756350632223),
 (1, 1967.3898954123326),
 (12, 6363.191583197213)]

In [36]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(31, 9077.582354244241),
 (50, 7294.908013760045),
 (38, 7263.022132245552),
 (14, 7118.820393266188),
 (12, 6363.191583197213),
 (49, 6338.835696860746),
 (35, 5085.810277533634),
 (22, 4771.02774348814),
 (41, 4678.662064399374),
 (6, 4366.0346757809475),
 (9, 4130.423699529387),
 (26, 4091.064078060395),
 (42, 3527.6591750609896),
 (28, 2968.277224797358),
 (17, 2904.8237462926286),
 (4, 2883.1745876816844),
 (10, 2821.334711698309),
 (25, 2757.706093868283),
 (47, 2616.1574928075424),
 (20, 2483.5366373268894),
 (29, 2460.63509167042),
 (27, 2452.6036567595643),
 (43, 2427.4265890629954),
 (46, 2417.3064052079912),
 (48, 2385.1241151333597),
 (32, 2303.845524947915),
 (15, 2275.7747938797374),
 (33, 2243.1262578107294),
 (21, 2220.756350632223),
 (23, 2141.65318175735),
 (44, 2135.2973440942847),
 (2, 2102.3632355107566),
 (8, 2049.1965284095186),
 (1, 1967.3898954123326),
 (40, 1791.5655048450471),
 (30, 1454.32432229845),
 (5, 1186.0393791786923)]

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [37]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(26, 15),
 (35, 13),
 (31, 12),
 (18, 12),
 (37, 11),
 (33, 11),
 (22, 9),
 (49, 9),
 (41, 9),
 (3, 9),
 (47, 8),
 (45, 8),
 (12, 8),
 (9, 7),
 (46, 7),
 (39, 7),
 (4, 6),
 (44, 6),
 (11, 6),
 (42, 5),
 (10, 5),
 (17, 5),
 (23, 5),
 (43, 5),
 (14, 5),
 (21, 5),
 (2, 5),
 (29, 5),
 (8, 5),
 (20, 5),
 (7, 4),
 (36, 4),
 (6, 4),
 (38, 4),
 (50, 4),
 (28, 4),
 (5, 3),
 (27, 3),
 (30, 3),
 (16, 3),
 (24, 3),
 (19, 3),
 (15, 2),
 (48, 2),
 (40, 2),
 (1, 2),
 (25, 1),
 (32, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
